# Basics
The basic notebook just imports the required dependencies and sets the ports and addresses correctly to be accessible outside of the Docker container. It then creates a link for viewing the data

In [1]:
from __future__ import print_function
from neuroglancer.server import global_server_args
# setup the binding so it is visible outside the container
global_server_args['bind_port'] = 8989
global_server_args['bind_address'] = '00.0.0.0' # TODO: the server renames addresses of 0.0.0.0 and they become unusable
import neuroglancer
import numpy as np

# 3D Numpy Data
Here we show 3D saved data in from a compressed numpy file (NPZ)
- __Note__ the data needs to be recast as np.uint32 type so it can be used in neuroglancer

In [2]:
with np.load('test_data.npz', 'r') as npz_file:
    raw_ct_image = npz_file['CT']
    ct_image = ((np.expand_dims(raw_ct_image,0)+1024).clip(0,2048)/2048.0).astype(np.float32) 
    raw_PET_image = npz_file['PET']
    pet_image = ((np.expand_dims(raw_PET_image,0)).clip(0,5)/5.0).astype(np.float32) 
    label_image = np.expand_dims(npz_file['Labels'].astype(np.float32),0)
    label_image /= label_image.max() # normalize labels
    vox_size = npz_file['spacing'] # in mm

# Create the viewer
Here the viewer is created with a simple shader to make the rendering colorful

In [3]:
neuroglancer.set_static_content_source(url='https://neuroglancer-demo.appspot.com')

viewer = neuroglancer.Viewer(voxel_size=1000*vox_size) # since vox_size
viewer.add(ct_image,
           name='CT Image')
viewer.add(pet_image,
           name='PET Image',
          shader = """
void main () {
  emitRGB(colormapJet(toNormalized(getDataValue())));
}
          """) 
# add as a solid red green with a varying degree of transparency
viewer.add(label_image,
           name='Label Image',
          shader = """
void main () {
  emitRGBA(vec4(0, 1.0, 0, toNormalized(getDataValue())));
}
          """) 

# Show the Results
Here when we show the viewer we get a link, altnatively the results can be shown using the IPython HTML displaying interface

In [4]:
viewer

http://00.0.0.0:8989/static/310f0b7b9fb783a8a0f1c6efb92505b16a78e7b1/#!%7B%22layers%22:%7B%22CT%20Image%22:%7B%22source%22:%22python://http://00.0.0.0:8989/9d1be81e302f5956bc04143432598a4fd9916bff%22,%22type%22:%22image%22%7D,%22PET%20Image%22:%7B%22source%22:%22python://http://00.0.0.0:8989/c01da334987dd01933ce670e6f3c9e7e491954ea%22,%22type%22:%22image%22,%22shader%22:%22%5Cnvoid%20main%20()%20%7B%5Cn%20%20emitRGB(colormapJet(toNormalized(getDataValue())));%5Cn%7D%5Cn%20%20%20%20%20%20%20%20%20%20%22%7D,%22Label%20Image%22:%7B%22source%22:%22python://http://00.0.0.0:8989/180b47babeb1854136ddced7fb57c78a92d8524b%22,%22type%22:%22image%22,%22shader%22:%22%5Cnvoid%20main%20()%20%7B%5Cn%20%20emitRGBA(vec4(1,%200,%200,%20toNormalized(getDataValue())));%5Cn%7D%5Cn%20%20%20%20%20%20%20%20%20%20%22%7D%7D,%22navigation%22:%7B%22pose%22:%7B%22voxelSize%22:%5B2976.5625,2976.5625,8000.0%5D%7D%7D%7D

In [5]:
from IPython.display import HTML
HTML(url = str(viewer))